In [1]:
import pandas as pd
import numpy as np
import os, pathlib
import librosa
from pathlib import Path
import glob
import pickle
import shutil
import datetime
import wave_manipulator


p_dsampled = Path.cwd() / 'data' / 'UIOWA' /'back'
p_temp1 = Path.cwd() / 'data' / 'UIOWA' /'back' / 'tmp1'
p_temp2 = Path.cwd() / 'data' / 'UIOWA' /'back' / 'tmp2'
p_temp3 = Path.cwd() / 'data' / 'UIOWA' /'back' / 'tmp3'
p_temp4 = Path.cwd() / 'data' / 'UIOWA' /'back' / 'tmp4'
p_temp5 = Path.cwd() / 'data' / 'UIOWA' /'back' / 'tmp5'
p_temp6 = Path.cwd() / 'data' / 'UIOWA' /'back' / 'tmp6'
p_temp6 = Path.cwd() / 'data' / 'UIOWA' /'back' / 'tmp7'
p_wavo = Path.cwd() / 'data' / 'UIOWA' /'back' / 'wavo'
p_wav = Path.cwd() / 'data' / 'UIOWA' /'back' / 'wav'
p_data = '/home/mirko/Documents/data/UIOWA/**'

### Define Parameters for Instruments
___

In [2]:
Instruments = {'Piano' : ['skip', 0.0, 0, 0], 'Guitar' : ['run', 0.1, 20, 33000], 'Viola' : ['run', 0.1, 20, 25000],
         'Violin' : ['run', 0.1, 20, 25000], 'Double Bass' : ['run', 0.1, 20, 25000], 'Cymbals' : ['skip', 0.0, 0, 0],
         'Oboe' : ['run', 0.01, 20, 22000], 'Bass Clarinet' : ['run', 0.01, 20, 22000], 'Bass Flute' : ['run', 0.01, 20, 22000],
         'Bb Clarinet' : ['run', 0.01, 20, 22000], 'Eb Alto Saxophone' : ['run', 0.01, 20, 22000], 'Alto Flute' : ['run', 0.01, 20, 22000],
         'Flute' : ['run', 0.01, 20, 44000], 'Eb Clarinet' : ['run', 0.01, 20, 22000], 'Bb Soprano Saxophone' : ['run', 0.01, 20, 22000],
         'Tuba' : ['run', 0.01, 20, 22000], 'Bass Trombone' : ['run', 0.01, 20, 22000], 'Tenor Trombone' : ['run', 0.01, 20, 22000],
         'Bassoon' : ['run', 0.01, 20, 22000], 'Bb Trumpet' : ['run', 0.01, 20, 22000], 'Marimba' : ['run', 0.01, 20, 4000],
         'Bells' : ['run', 0.01, 20, 4000], 'Crotales' : ['skip', 0.0, 0, 0], 'Cymbals' : ['skip', 0.0, 0, 0], 'Gongs' : ['skip', 0.0, 0, 0],
         'Hand Percussion' : ['skip', 0.0, 0, 0], 'Tamtam' : ['skip', 0.0, 0, 0], 'Tambourines' : ['skip', 0.0, 0, 0],
         'Vibraphone' : ['run', 0.01, 20, 34200], 'Xylophone' : ['run', 0.01, 20, 22000], 'Cello' : ['run', 0.1, 20, 25000]
        }

### Read and structure raw files from disc
___

In [ ]:
pathframe = wave_manipulator.create_pathfile(p_data) #Read the filestructure and dump to disc
wave_manipulator.create_rawfiles2(pathframe, p_temp1)

### Slice and trim the takeouts
___

In [ ]:

for instrument, value in Instruments.items():
    if value[0] == 'skip':
        shutil.copyfile(p_dsampled / (instrument + '.pkl'), p_temp2 / (instrument + '.pkl'))
    else:
        print(instrument, value)
        data = pd.read_pickle(p_dsampled / (instrument + '.pkl'))
        for i in range(60, 20, -1):
            data = wave_manipulator.slice_recording2(data, Ignore=132000, min_loud=value[1], top_db=i, no_samples=value[3])
            data = wave_manipulator.delete_trash(data, length=25000, top_db=100)
        data.to_pickle(p_temp2 / (instrument + '.pkl'))

### Iterate to trim until minimum value reached
___

In [ ]:
for instrument, value in Instruments.items():
    data = pd.read_pickle(p_temp2 / (instrument + '.pkl'))
    
    for i in range(120, 20, -1):
        n = i/2
        data = wave_manipulator.trim_silence2(data, Ignore=44100*3, top_db=n)
        
data.to_pickle(p_temp3 / (instrument + '.pkl'))

### Slice trailing end over given length
___

In [3]:
for instrument, value in Instruments.items():
    data = pd.read_pickle(p_temp3 / (instrument + '.pkl'))
    data = wave_manipulator.cut_end(data, length=132300)
    data.to_pickle(p_temp4 / (instrument + '.pkl'))

### Clean the edges
___

In [4]:
for instrument, value in Instruments.items():
    data = pd.read_pickle(p_temp4 / (instrument + '.pkl'))
    data = wave_manipulator.clean_edges(data, length=132300)
    data.to_pickle(p_temp5 / (instrument + '.pkl'))

### Once again drop too silent recordings
___

In [5]:
for instrument, value in Instruments.items():
    data = pd.read_pickle(p_temp5 / (instrument + '.pkl'))
    data = wave_manipulator.delete_trash(data, length=200000, top_db=0.01)
    data.to_pickle(p_temp6 / (instrument + '.pkl'))

### Drop to wave for Inspection
___

In [6]:
for instrument, value in Instruments.items():
#     data = pd.read_pickle(p_dsampled / (instrument + '.pkl'))
#     wave_manipulator.to_wav3(data, p=p_wav/(instrument+'_Original'), sr=44100)
    data = pd.read_pickle(p_temp4 / (instrument + '.pkl'))
    wave_manipulator.to_wav3(data, p=p_wav/(instrument+'_Sliced'), sr=44100)
    data = pd.read_pickle(p_temp5 / (instrument + '.pkl'))
    wave_manipulator.to_wav3(data, p=p_wav/(instrument+'_Dropped again'), sr=44100)
    data = pd.read_pickle(p_temp6 / (instrument + '.pkl'))
    wave_manipulator.to_wav3(data, p=p_wav/(instrument+'_Clean'), sr=44100)

12:44:09 : Creating Pandas Frame with path and labels


In [17]:
instrument = 'Piano'
x, sr = librosa.load(p_wav / (instrument + '_Clean') / '41_1.wav',  sr=None)
print(np.max(x))
x[0:12]
z = librosa.zero_crossings(x)
crossings = np.nonzero(z)
crossings[0][-1]

0.0043640137


132186